In [1]:
from lib.outliers import Runner
from datetime import date

In [2]:
from_date = date(year=2021,month=4,day=1)
to_date = date(year=2021,month=8,day=1)
r = Runner(from_date,to_date,5,["practice","ccg","pcn","stp"],False)

In [3]:
r.run()

Downloading: 100%|██████████| 1/1 [00:00<00:00,  6.92rows/s]


TypeError: add_item() argument after ** must be a mapping, not BrokenProcessPool